## Exercises: Lookups

In the following exercises you will need to use the following terms to make the expressions typecheck:

1. `pure`
2. `(<$>)`
3. `(<*>)`

In [27]:
-- 1.

added :: Maybe Integer
added = (+3) <$> (lookup 3 $ zip [1, 2, 3] [4, 5, 6])

added

-- 2.

y :: Maybe Integer
y = lookup 3 $ zip [1, 2, 3] [4, 5, 6]

z :: Maybe Integer
z = lookup 2 $ zip [1, 2, 3] [4, 5, 6]

tupled :: Maybe (Integer, Integer)
tupled = (,) <$> y <*> z

tupled

-- 3.

import Data.List (elemIndex)

x :: Maybe Int
x = elemIndex 3 [1, 2, 3, 4, 5]

y :: Maybe Int
y = elemIndex 4 [1, 2, 3, 4, 5]

max' :: Int -> Int -> Int
max' = max

maxed :: Maybe Int
maxed = max' <$> x <*> y

maxed

-- 4.

xs = [1, 2, 3]
ys = [4, 5, 6]

x :: Maybe Integer
x = lookup 3 $ zip xs ys

y :: Maybe Integer
y = lookup 2 $ zip xs ys

summed :: Maybe Integer
summed = fmap sum $ (,) <$> x <*> y

Just 9

Just (6,5)

Just 3

## Exercise: Identity Instance

Write an `Applicative` instance for `Identity`.

In [29]:
newtype Identity a =
    Identity a
    deriving (Eq, Ord, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

instance Applicative Identity where
    pure = Identity
    (<*>) (Identity f) (Identity a) = Identity (f a)

## Exercise: Constant Instance

Write an `Applicative` instance for `Constant`.

In [31]:
newtype Constant a b =
    Constant { getConstant :: a }
    deriving (Eq, Ord, Show)

instance Functor (Constant a) where
    fmap f (Constant b) = Constant b

instance Monoid a => Applicative (Constant a) where
    pure _ = Constant { getConstant = mempty }
    (<*>) (Constant a) (Constant b) = Constant (mappend a b)

## Exercise: Fixer Upper

Given the function and values provided, use (`<$>`) from `Functor`, (`<*>`) and pure from the `Applicative` typeclass to fill in missing bits of the broken code to make it work.

In [8]:
-- 1.

const <$> Just "Hello" <*> Just "World"

-- 2.

(,,,) <$> Just 90 <*> Just 10 <*> Just "Tierness" <*> Just [1, 2, 3]

Just "Hello"

Just (90,10,"Tierness",[1,2,3])

## Exercise: List Applicative

Implement the list `Applicative`. Writing a minimally complete `Applicative` instance calls for writing the definitions of both `pure` and `<*>`. We’re going to provide a hint as well. Use the `checkers` library to validate your `Applicative` instance.

In [30]:
data List a =
    Nil
    | Cons a (List a)
    deriving (Eq, Show)

append :: List a -> List a -> List a
append Nil ys         = ys
append (Cons x xs) ys =
    Cons x $ xs `append` ys

fold :: (a -> b -> b) -> b -> List a -> b
fold _ b Nil        = b
fold f b (Cons h t) = f h (fold f b t)

concat' :: List (List a) -> List a
concat' = fold append Nil

flatMap :: (a -> List b) -> List a -> List b
flatMap f as = concat' $ fmap f as

instance Functor List where
    fmap _ Nil         = Nil
    fmap f (Cons x xs) =
        Cons (f x) (fmap f xs)
    
instance Applicative List where
    pure x               = Cons x Nil
    (<*>) Nil _          = Nil
    (<*>) _ Nil          = Nil
    (<*>) (Cons f fs) xs =
        append (fmap f xs) (fs <*> xs)
    
instance Applicative List where
  pure x               = Cons x Nil
  (<*>) Nil _          = Nil
  (<*>) _ Nil          = Nil
  (<*>) (Cons f fs) xs = append (fmap f xs) (fs <*> xs)

--

f = Cons (+1) (Cons (*2) Nil)
v = Cons 1 (Cons 2 Nil)
f <*> v

toMyList = foldr Cons Nil
xs = toMyList [1, 2, 3]
c = Cons
f x = x `c` (9 `c` Nil)
flatMap f xs

Cons 2 (Cons 3 (Cons 2 (Cons 4 Nil)))

Cons 1 (Cons 9 (Cons 2 (Cons 9 (Cons 3 (Cons 9 Nil)))))